# Player Tracking System Comparison

This notebook compares 3 different player tracking systems:
- Eagle
- Darkmyter (using Ultralytics YOLO)
- TrackLab

**Important**: Run cells in order from top to bottom!

In [1]:
# Cell 1: Setup directories and utilities

from pathlib import Path
import os

BASE_DIR = Path("/content")
REPOS_DIR = BASE_DIR / "repositories"
VIDEOS_DIR = BASE_DIR / "videos"
CLIPS_DIR = BASE_DIR / "clips"
OUTPUT_DIR = BASE_DIR / "output"

for d in [REPOS_DIR, VIDEOS_DIR, CLIPS_DIR, OUTPUT_DIR]:
    d.mkdir(parents=True, exist_ok=True)

def print_status(msg, status="INFO"):
    """Print colored status messages"""
    colors = {
        "INFO": "\033[94m",
        "SUCCESS": "\033[92m",
        "WARNING": "\033[93m",
        "ERROR": "\033[91m",
        "RESET": "\033[0m"
    }
    print(f"{colors.get(status, '')}[{status}] {msg}{colors['RESET']}")

print_status("Directory structure created", "SUCCESS")
print(f"Working directory: {BASE_DIR}")

[SUCCESS] Directory structure created
Working directory: /content


In [2]:
# Cell 2: Clone all repositories

import subprocess

REPOSITORIES = {
    "eagle": "https://github.com/nreHieW/Eagle.git",
    "darkmyter": "https://github.com/Darkmyter/Football-Players-Tracking.git",
    "tracklab": "https://github.com/TrackingLaboratory/tracklab.git"
}

print_status("Cloning repositories...", "INFO")

for name, url in REPOSITORIES.items():
    repo_path = REPOS_DIR / name

    if repo_path.exists():
        print_status(f"{name}: Already exists, skipping", "WARNING")
        continue

    try:
        print_status(f"{name}: Cloning...", "INFO")
        result = subprocess.run(
            ["git", "clone", url, str(repo_path)],
            capture_output=True,
            text=True,
            timeout=300
        )

        if result.returncode == 0:
            print_status(f"{name}: Cloned successfully", "SUCCESS")
        else:
            print_status(f"{name}: Clone failed - {result.stderr[:100]}", "ERROR")

    except Exception as e:
        print_status(f"{name}: Clone failed - {str(e)}", "ERROR")

print_status("Repository cloning complete", "SUCCESS")

[INFO] Cloning repositories...
[INFO] eagle: Cloning...
[SUCCESS] eagle: Cloned successfully
[INFO] darkmyter: Cloning...
[SUCCESS] darkmyter: Cloned successfully
[INFO] tracklab: Cloning...
[SUCCESS] tracklab: Cloned successfully
[SUCCESS] Repository cloning complete


In [ ]:
# Cell 3: Install dependencies

print_status("Installing dependencies...", "INFO")

!pip install -q torch torchvision torchaudio
!pip install -q opencv-python numpy scipy pandas scikit-learn matplotlib
!pip install -q ultralytics supervision
!pip install -q gdown Pillow tqdm requests

print_status("Dependencies installed", "SUCCESS")

[INFO] Installing dependencies...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 76.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.2/207.2 kB 17.8 MB/s eta 0:00:00
[SUCCESS] Dependencies installed


In [3]:
# Cell 4: Download videos from Google Drive

!pip install -q gdown

import gdown
from pathlib import Path

# Shared folder ID
FOLDER_ID = "1Cs4kTX6GYwfcpKyDZdqRKBezz49wT7_N"

print_status("Downloading videos from shared folder...", "INFO")

try:
    gdown.download_folder(
        id=FOLDER_ID,
        output=str(VIDEOS_DIR),
        quiet=False,
        use_cookies=False
    )

    # List downloaded videos
    video_extensions = ['.mp4', '.avi', '.mov', '.mkv', '.MP4', '.AVI', '.MOV', '.MKV']
    available_videos = []

    for ext in video_extensions:
        available_videos.extend(list(VIDEOS_DIR.glob(f"*{ext}")))

    if not available_videos:
        print_status("No video files found", "ERROR")
    else:
        print(f"DOWNLOADED {len(available_videos)} VIDEO(S)")


        for idx, video in enumerate(available_videos, 1):
            size_mb = video.stat().st_size / (1024 * 1024)
            print(f"{idx}. {video.name} ({size_mb:.1f} MB)")


        print("Enter video selection:")
        print("  - Leave blank to process ALL videos")
        print("  - Enter a number (e.g., '1')")
        print("  - Enter comma-separated numbers (e.g., '1,2')")

        selection = input("\nYour choice: ").strip()

        VIDEO_PATHS = []

        if not selection:
            VIDEO_PATHS = available_videos
            print_status(f"Selected ALL {len(VIDEO_PATHS)} videos", "SUCCESS")
        elif selection.isdigit():
            idx = int(selection)
            if 1 <= idx <= len(available_videos):
                VIDEO_PATHS = [available_videos[idx - 1]]
                print_status(f"Selected: {VIDEO_PATHS[0].name}", "SUCCESS")
        elif ',' in selection:
            try:
                indices = [int(x.strip()) for x in selection.split(',')]
                for idx in indices:
                    if 1 <= idx <= len(available_videos):
                        VIDEO_PATHS.append(available_videos[idx - 1])
                print_status(f"Selected {len(VIDEO_PATHS)} videos", "SUCCESS")
            except ValueError:
                print_status("Invalid input", "ERROR")

        if not VIDEO_PATHS:
            print_status("No videos selected", "ERROR")

except Exception as e:
    print_status(f"Download failed: {str(e)}", "ERROR")
    print("\nNote: Make sure the folder is set to 'Anyone with the link can view'")

[INFO] Downloading videos from shared folder...


Retrieving folder contents


Processing file 1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
Processing file 1RvqkxASOD23jfigqSgSgGja5_NGZReO4 FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
Processing file 1urwKF6wjitkREymiNi9O3jCLLIysTp6F FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4


Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From (original): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf
From (redirected): https://drive.google.com/uc?id=1uXckJCK4pVPfoRvJWaZmtM_uH6pFQogf&confirm=t&uuid=7a285f3b-3872-4069-aeef-06c9bd90bd2b
To: /content/videos/FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4
100%|██████████| 1.68G/1.68G [00:23<00:00, 70.4MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4
From (redirected): https://drive.google.com/uc?id=1RvqkxASOD23jfigqSgSgGja5_NGZReO4&confirm=t&uuid=136a8d3d-5ef5-44e5-b2a8-4225d42dde48
To: /content/videos/FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4
100%|██████████| 1.92G/1.92G [00:23<00:00, 80.2MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jCLLIysTp6F
From (redirected): https://drive.google.com/uc?id=1urwKF6wjitkREymiNi9O3jC

DOWNLOADED 3 VIDEO(S)
1. FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4 (1260.8 MB)
2. FULL MATCH  Brazil v Mexico  World Cup 2018 720p.mp4 (1832.4 MB)
3. FULL MATCH  Belgium 1-2 Italy  VIP Tactical Camera 720.mp4 (1604.1 MB)
Enter video selection:
  - Leave blank to process ALL videos
  - Enter a number (e.g., '1')
  - Enter comma-separated numbers (e.g., '1,2')

Your choice: 1
[SUCCESS] Selected: FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p.mp4


In [4]:
# Cell 5: Extract clips

import cv2

CLIP_DURATION = 60
ALL_CLIPS = {}

for VIDEO_PATH in VIDEO_PATHS:
    VIDEO_NAME = VIDEO_PATH.stem


    print(f"PROCESSING: {VIDEO_NAME}\n")


    cap = cv2.VideoCapture(str(VIDEO_PATH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    duration = total_frames / fps
    cap.release()

    print(f"Duration: {duration:.1f}s | FPS: {fps:.1f} | Frames: {total_frames}")

    if duration < CLIP_DURATION * 3:
        if duration < CLIP_DURATION:
            CLIPS = [(0, duration, "full")]
        else:
            CLIPS = [(0, CLIP_DURATION, "start"), (max(duration - CLIP_DURATION, 0), CLIP_DURATION, "end")]
    else:
        CLIPS = [
            (0, CLIP_DURATION, "start"),
            ((duration - CLIP_DURATION) / 2, CLIP_DURATION, "middle"),
            (duration - CLIP_DURATION, CLIP_DURATION, "end")
        ]

    CLIP_PATHS = {}

    for start_time, clip_dur, position in CLIPS:
        clip_name = f"{VIDEO_NAME}_{position}.mp4"
        clip_path = CLIPS_DIR / clip_name

        cmd = ["ffmpeg", "-i", str(VIDEO_PATH), "-ss", str(start_time), "-t", str(clip_dur),
               "-c", "copy", str(clip_path), "-y", "-loglevel", "error"]

        result = subprocess.run(cmd, capture_output=True)

        if result.returncode == 0 and clip_path.exists():
            CLIP_PATHS[position] = clip_path
            print_status(f"Clip '{position}' extracted", "SUCCESS")

    ALL_CLIPS[VIDEO_NAME] = CLIP_PATHS

print(f"\nTotal: {sum(len(clips) for clips in ALL_CLIPS.values())} clips from {len(VIDEO_PATHS)} video(s)")

PROCESSING: FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p

Duration: 5967.2s | FPS: 25.0 | Frames: 149181
[SUCCESS] Clip 'start' extracted
[SUCCESS] Clip 'middle' extracted
[SUCCESS] Clip 'end' extracted

Total: 3 clips from 1 video(s)


In [ ]:
# ================================
# Cell: Setup Eagle - Corrected
# ================================

print_status("Setting up Eagle...", "INFO")

eagle_dir = REPOS_DIR / "eagle"

# Install Python 3.11 (required for Eagle's PyTorch version)
print_status("Installing Python 3.11...", "INFO")
!apt-get update -qq
!apt-get install -y python3.11 python3.11-venv python3.11-dev

# Install uv if not already installed
print_status("Installing uv...", "INFO")
!curl -LsSf https://astral.sh/uv/install.sh | sh

# Add uv to PATH
import os
os.environ['PATH'] = f"/root/.local/bin:{os.environ['PATH']}"

# Download model weights
print_status("Downloading Eagle model weights...", "INFO")
models_dir = eagle_dir / "eagle" / "models"
if models_dir.exists():
    os.chdir(models_dir)
    !bash get_weights.sh
    os.chdir(BASE_DIR)
    print_status("Eagle weights downloaded", "SUCCESS")
else:
    print_status("Eagle models directory not found", "ERROR")

# Create corrected Eagle wrapper
eagle_wrapper = eagle_dir / "run_eagle.py"
eagle_wrapper.write_text('''
import argparse
import json
import subprocess
import sys
from pathlib import Path
import cv2

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True, help="Input video path")
    parser.add_argument("--output", required=True, help="Output JSON path")
    parser.add_argument("--fps", default=24, type=int, help="FPS for processing")
    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: Video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    print(f"Processing video: {video_path}", file=sys.stderr)
    print(f"Output will be saved to: {output_path}", file=sys.stderr)

    # Run Eagle using uv with Python 3.11
    cmd = [
        "uv", "run", "--python", "python3.11",
        "main.py",
        "--video_path", str(video_path),
        "--fps", str(args.fps),
        "--track_players_only"  # Skip ball detection to avoid errors
    ]

    print(f"Running Eagle...", file=sys.stderr)

    result = subprocess.run(
        cmd,
        capture_output=True,
        text=True,
        cwd=Path(__file__).parent,
        timeout=600  # 10 minute timeout
    )

    # Eagle creates output in its own directory structure
    video_stem = video_path.stem
    eagle_output_dir = Path(__file__).parent / "output" / video_stem

    # Try to find and convert Eagle's output
    tracking_data = extract_eagle_tracking(eagle_output_dir, result)

    if tracking_data:
        # Save in evaluation format
        output_path.parent.mkdir(parents=True, exist_ok=True)
        with open(output_path, 'w') as f:
            json.dump(tracking_data, f, indent=2)

        print(f"Saved {len(tracking_data)} detections", file=sys.stderr)
    else:
        print("Failed to extract tracking data", file=sys.stderr)
        sys.exit(1)

def extract_eagle_tracking(eagle_output_dir, result):
    """Extract and convert Eagle's tracking output to evaluation format"""

    # Look for Eagle's tracking files
    if not eagle_output_dir.exists():
        print(f"Eagle output directory not found: {eagle_output_dir}", file=sys.stderr)
        return None

    # Try to find JSON tracking data
    json_files = list(eagle_output_dir.glob("*.json"))
    csv_files = list(eagle_output_dir.glob("*.csv"))

    all_detections = []

    # First try JSON files
    for json_file in json_files:
        try:
            with open(json_file, 'r') as f:
                data = json.load(f)

            # Convert Eagle's format to evaluation format
            if isinstance(data, dict):
                # Frame-indexed format
                for frame_str, detections in data.items():
                    frame_id = int(frame_str)

                    if isinstance(detections, list):
                        for det in detections:
                            # Eagle might use different field names
                            track_id = det.get('id', det.get('track_id', det.get('player_id', 0)))

                            # Convert bbox format if needed
                            if 'x' in det and 'y' in det and 'w' in det and 'h' in det:
                                # Convert from xywh to xyxy
                                x, y, w, h = det['x'], det['y'], det['w'], det['h']
                                x1, y1 = x - w/2, y - h/2
                                x2, y2 = x + w/2, y + h/2
                                bbox = [x1, y1, x2, y2]
                            elif 'bbox' in det:
                                bbox = det['bbox']
                            elif 'x1' in det and 'y1' in det and 'x2' in det and 'y2' in det:
                                bbox = [det['x1'], det['y1'], det['x2'], det['y2']]
                            else:
                                continue  # Skip if no valid bbox

                            all_detections.append({
                                "frame_id": frame_id,
                                "track_id": int(track_id),
                                "bbox": [float(b) for b in bbox],
                                "score": float(det.get('confidence', det.get('score', 1.0))),
                                "class_id": 0  # Person class
                            })

            elif isinstance(data, list):
                # Already in list format
                for det in data:
                    if 'frame_id' in det or 'frame' in det:
                        frame_id = det.get('frame_id', det.get('frame', 0))
                        track_id = det.get('track_id', det.get('id', 0))

                        # Handle bbox format
                        if 'bbox' in det:
                            bbox = det['bbox']
                        elif 'x' in det and 'y' in det and 'w' in det and 'h' in det:
                            x, y, w, h = det['x'], det['y'], det['w'], det['h']
                            bbox = [x - w/2, y - h/2, x + w/2, y + h/2]
                        else:
                            continue

                        all_detections.append({
                            "frame_id": int(frame_id),
                            "track_id": int(track_id),
                            "bbox": [float(b) for b in bbox],
                            "score": float(det.get('confidence', det.get('score', 1.0))),
                            "class_id": 0
                        })

            if all_detections:
                print(f"Extracted {len(all_detections)} detections from {json_file.name}", file=sys.stderr)
                return all_detections

        except Exception as e:
            print(f"Failed to parse {json_file}: {e}", file=sys.stderr)
            continue

    # Try CSV files if no JSON worked
    if csv_files and not all_detections:
        try:
            import pandas as pd

            for csv_file in csv_files:
                df = pd.read_csv(csv_file)

                # Convert CSV to our format
                for _, row in df.iterrows():
                    frame_id = int(row.get('frame', row.get('frame_id', 0)))
                    track_id = int(row.get('id', row.get('track_id', row.get('player_id', 0))))

                    # Get bbox
                    if 'x1' in row and 'y1' in row and 'x2' in row and 'y2' in row:
                        bbox = [row['x1'], row['y1'], row['x2'], row['y2']]
                    elif 'x' in row and 'y' in row and 'w' in row and 'h' in row:
                        x, y, w, h = row['x'], row['y'], row['w'], row['h']
                        bbox = [x - w/2, y - h/2, x + w/2, y + h/2]
                    else:
                        continue

                    all_detections.append({
                        "frame_id": frame_id,
                        "track_id": track_id,
                        "bbox": [float(b) for b in bbox],
                        "score": float(row.get('confidence', row.get('score', 1.0))),
                        "class_id": 0
                    })

                if all_detections:
                    print(f"Extracted {len(all_detections)} detections from {csv_file.name}", file=sys.stderr)
                    return all_detections

        except Exception as e:
            print(f"Failed to parse CSV files: {e}", file=sys.stderr)

    # If still no data, create empty output
    if not all_detections:
        print("No valid tracking data found in Eagle output", file=sys.stderr)
        return []

    return all_detections

if __name__ == "__main__":
    main()
''')

eagle_wrapper.chmod(0o755)

# Test the setup
print_status("Testing Eagle setup...", "INFO")

test_cmd = ["python", str(eagle_wrapper), "--help"]
try:
    result = subprocess.run(test_cmd, capture_output=True, text=True, cwd=eagle_dir)
    if result.returncode == 0:
        print_status("✓ Eagle wrapper script works", "SUCCESS")
    else:
        print_status("✗ Eagle wrapper failed", "ERROR")
        print(result.stderr)
except Exception as e:
    print_status(f"✗ Eagle test failed: {e}", "ERROR")

print_status("Eagle setup complete", "SUCCESS")

[INFO] Setting up Eagle...
[INFO] Installing Python 3.11...
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libpython3.11 libpython3.11-dev libpython3.11-minimal libpython3.11-stdlib
  python3.11-minimal
Suggested packages:
  binfmt-support
The following NEW packages will be installed:
  libpython3.11 libpython3.11-dev libpython3.11-minimal libpython3.11-stdlib
  python3.11 python3.11-dev python3.11-minimal python3.11-venv
0 upgraded, 8 newly installed, 0 to remove and 46 not upgraded.
Need to get 16.5 MB of archives.
After this operation, 58.4 MB of additional disk space will be used.
Get:1 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 libpython3.11-minimal amd64 3.11.14-

In [ ]:

# Cell: Setup Darkmyter (ByteTrack + YOLO)

print_status("Setting up Darkmyter tracking...", "INFO")

import os
import subprocess
from pathlib import Path

darkmyter_dir = REPOS_DIR / "darkmyter"

# Download football-specific weights
weights_dir = darkmyter_dir / "yolov8-weights"
weights_dir.mkdir(parents=True, exist_ok=True)

custom_weights = weights_dir / "yolov8l-football-players.pt"
gdrive_id = "12dWRBsegmyGE3feTdy9LBf1eZ-hTZ9Sx"

def download_darkmyter_weights():
    print_status("Downloading Darkmyter football weights...", "INFO")
    try:
        try:
            import gdown
        except ImportError:
            subprocess.run(["pip", "install", "gdown"], check=True)
            import gdown

        url = f"https://drive.google.com/uc?id={gdrive_id}"
        gdown.download(url, str(custom_weights), quiet=False)
        print_status("Darkmyter weights downloaded", "SUCCESS")
    except Exception as e:
        print_status(f"Failed to download weights: {e}", "ERROR")

# Check if weights exist and are valid
if custom_weights.exists():
    try:
        with open(custom_weights, "rb") as f:
            header = f.read(16)
        if header.startswith(b"<"):
            print_status("Weights file is HTML, re-downloading...", "ERROR")
            custom_weights.unlink(missing_ok=True)
            download_darkmyter_weights()
        else:
            print_status("Darkmyter weights already present", "SUCCESS")
    except Exception:
        custom_weights.unlink(missing_ok=True)
        download_darkmyter_weights()
else:
    download_darkmyter_weights()

# Create corrected Darkmyter wrapper
darkmyter_wrapper = darkmyter_dir / "run_darkmyter.py"
darkmyter_wrapper.write_text('''
import argparse
import json
from pathlib import Path
import sys

try:
    from ultralytics import YOLO
except ImportError:
    print("Error: ultralytics not installed. Run: pip install ultralytics", file=sys.stderr)
    sys.exit(1)

def load_model(repo_root: Path):
    """Load Darkmyter's football-specific weights or fallback to generic."""
    custom_weights = repo_root / "yolov8-weights" / "yolov8l-football-players.pt"

    if custom_weights.exists():
        try:
            print(f"Loading Darkmyter football weights: {custom_weights}", file=sys.stderr)
            return YOLO(str(custom_weights))
        except Exception as e:
            print(f"Failed to load custom weights: {e}", file=sys.stderr)
            print("Falling back to yolov8x.pt", file=sys.stderr)
            return YOLO("yolov8x.pt")
    else:
        print("Using generic yolov8x.pt (custom weights not found)", file=sys.stderr)
        return YOLO("yolov8x.pt")

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True, help="Input video path")
    parser.add_argument("--output", required=True, help="Output JSON path")
    parser.add_argument("--conf", type=float, default=0.3, help="Confidence threshold")
    parser.add_argument("--iou", type=float, default=0.5, help="IoU threshold")

    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)

    if not video_path.exists():
        print(f"Error: Video not found: {video_path}", file=sys.stderr)
        sys.exit(1)

    repo_root = Path(__file__).resolve().parent
    model = load_model(repo_root)

    print(f"Processing: {video_path}", file=sys.stderr)
    print(f"Output: {output_path}", file=sys.stderr)

    # Run tracking with ByteTrack
    results = model.track(
        source=str(video_path),
        tracker="bytetrack.yaml",
        conf=args.conf,
        iou=args.iou,
        persist=True,
        verbose=False,
        device='cuda' if torch.cuda.is_available() else 'cpu'
    )

    # Collect all detections in the evaluation format
    all_detections = []

    for frame_idx, result in enumerate(results):
        if result.boxes is not None and result.boxes.id is not None:
            # Get xyxy format (corners) instead of xywh
            boxes_xyxy = result.boxes.xyxy.cpu().numpy()
            track_ids = result.boxes.id.cpu().numpy()
            confs = result.boxes.conf.cpu().numpy()
            classes = result.boxes.cls.cpu().numpy() if result.boxes.cls is not None else [0] * len(track_ids)

            for box, track_id, conf, cls in zip(boxes_xyxy, track_ids, confs, classes):
                all_detections.append({
                    "frame_id": int(frame_idx),
                    "track_id": int(track_id),
                    "bbox": [float(box[0]), float(box[1]), float(box[2]), float(box[3])],  # x1, y1, x2, y2
                    "score": float(conf),
                    "class_id": int(cls)  # Usually 0 for person
                })

        # Progress indicator
        if frame_idx % 100 == 0:
            print(f"Processed {frame_idx} frames...", file=sys.stderr)

    # Save output in the expected format
    output_path.parent.mkdir(parents=True, exist_ok=True)
    with output_path.open("w") as f:
        json.dump(all_detections, f, indent=2)

    print(f"Saved {len(all_detections)} detections from {frame_idx + 1} frames", file=sys.stderr)

if __name__ == "__main__":
    import torch  # Import here to check CUDA availability
    main()
''')

darkmyter_wrapper.chmod(0o755)

# Test the setup
print_status("Testing Darkmyter setup...", "INFO")

test_cmd = ["python", str(darkmyter_wrapper), "--help"]
try:
    result = subprocess.run(test_cmd, capture_output=True, text=True, cwd=darkmyter_dir)
    if result.returncode == 0:
        print_status("✓ Darkmyter wrapper script works", "SUCCESS")
    else:
        print_status("✗ Darkmyter wrapper failed", "ERROR")
        print(result.stderr)
except Exception as e:
    print_status(f"✗ Darkmyter test failed: {e}", "ERROR")

print_status("Darkmyter setup complete", "SUCCESS")

In [11]:

# Cell: Complete TrackLab Setup


import os
import sys
import json
import yaml
import shutil
import subprocess
import textwrap
from pathlib import Path

print_status("Setting up TrackLab with proper integration...", "INFO")

# --- Virtual display for OpenCV / matplotlib ---
!apt-get update -qq > /dev/null 2>&1
!apt-get install -y xvfb > /dev/null 2>&1
!pip install pyvirtualdisplay > /dev/null 2>&1

from pyvirtualdisplay import Display
display = Display(visible=0, size=(1400, 900))
display.start()
print_status("Virtual display started", "SUCCESS")

tracklab_dir = REPOS_DIR / "tracklab"
venv_dir = tracklab_dir / ".venv"

# --- Clean slate: remove old .venv if it exists ---
if venv_dir.exists():
    print_status("Removing existing TrackLab .venv...", "INFO")
    shutil.rmtree(venv_dir)

# --- Create uv environment inside the repo ---
os.chdir(tracklab_dir)
print_status("Creating TrackLab uv environment...", "INFO")
!uv venv --python 3.12

# --- Install TrackLab with ALL necessary dependencies ---
print_status("Installing TrackLab with video extras and dependencies...", "INFO")
!uv pip install -q -e ".[video]"
!uv pip install -q pyyaml
!uv pip install -q ultralytics  # YOLOv8
!uv pip install -q yolov5  # YOLOv5
!uv pip install -q pandas numpy opencv-python

# --- Apply necessary fixes to TrackLab ---
print_status("Applying fixes to TrackLab...", "INFO")

progress_path = tracklab_dir / "tracklab" / "callbacks" / "progress.py"
video_py_path = tracklab_dir / "tracklab" / "engine" / "video.py"

# Fix 1: Progress callback - handle missing 'name' field
!sed -i "s/video_metadata\\['name'\\]/video_metadata.get('name', 'unknown')/g" "{progress_path}"

# Fix 2: Progress callback - guard missing task progress bars
with open(progress_path, "r") as f:
    lines = f.readlines()

for i, line in enumerate(lines):
    if "self.task_pbars[task].update()" in line and i > 0 and "if task in" not in lines[i - 1]:
        indent = len(line) - len(line.lstrip())
        lines[i] = (
            " " * indent + "if task in self.task_pbars:\n"
            + " " * (indent + 4) + "self.task_pbars[task].update()\n"
        )
        break

with open(progress_path, "w") as f:
    f.writelines(lines)

# Fix 3 & 4: Video engine fixes
with open(video_py_path, "r") as f:
    content = f.read()

# Add image_pred parameter
old_callback = """        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
        )"""

new_callback = """        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
            image_pred=pd.DataFrame(),
        )"""

if old_callback in content:
    content = content.replace(old_callback, new_callback)

# Add tracker_state context manager
old_code = """    def track_dataset(self):
        \"\"\"Run tracking on complete dataset.\"\"\"
        self.callback("on_dataset_track_start")
        self.callback(
            "on_video_loop_start",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            index=0,
        )
        detections = self.video_loop()
        self.callback(
            "on_video_loop_end",
            video_metadata=pd.Series(name=self.video_filename),
            video_idx=0,
            detections=detections,
            image_pred=pd.DataFrame(),
        )
        self.callback("on_dataset_track_end")
        return detections"""

new_code = """    def track_dataset(self):
        \"\"\"Run tracking on complete dataset.\"\"\"
        self.callback("on_dataset_track_start")

        # Use tracker_state context manager for proper save functionality
        video_id = 0
        with self.tracker_state(video_id):
            self.callback(
                "on_video_loop_start",
                video_metadata=pd.Series(name=self.video_filename),
                video_idx=0,
                index=0,
            )
            detections = self.video_loop()
            self.callback(
                "on_video_loop_end",
                video_metadata=pd.Series(name=self.video_filename),
                video_idx=0,
                detections=detections,
                image_pred=pd.DataFrame(),
            )

        self.callback("on_dataset_track_end")
        return detections"""

if old_code in content:
    content = content.replace(old_code, new_code)

with open(video_py_path, "w") as f:
    f.write(content)

print_status("TrackLab fixes applied", "SUCCESS")


# Create TrackLab runner script (Proper implementation)


print_status("Creating proper TrackLab runner script...", "INFO")

runner_script = tracklab_dir / "run_tracklab.py"
runner_script.write_text(textwrap.dedent('''
#!/usr/bin/env python
"""
Proper TrackLab runner that uses the framework's modular architecture.
Compatible with the evaluation pipeline.
"""
import argparse
import json
import sys
from pathlib import Path

def main():
    parser = argparse.ArgumentParser()
    parser.add_argument("--video", required=True, help="Input video path")
    parser.add_argument("--output", required=True, help="Output JSON path")
    parser.add_argument("--detector", default="yolov8", help="Detector type")
    parser.add_argument("--tracker", default="bytetrack", help="Tracker type")
    parser.add_argument("--conf_threshold", type=float, default=0.3)
    args = parser.parse_args()

    video_path = Path(args.video)
    output_path = Path(args.output)
    output_path.parent.mkdir(parents=True, exist_ok=True)

    # Map detector names to model files
    detector_models = {
        "yolov8": "yolov8n.pt",
        "yolov8n": "yolov8n.pt",
        "yolov8s": "yolov8s.pt",
        "yolov8m": "yolov8m.pt",
        "yolov5": "yolov5s.pt",
        "yolov5s": "yolov5s.pt",
    }

    model_name = detector_models.get(args.detector, "yolov8n.pt")

    # Try using ultralytics with built-in tracking
    try:
        from ultralytics import YOLO
        import cv2
        import numpy as np

        print(f"Using {model_name} with {args.tracker} tracker", file=sys.stderr)

        # Load model
        model = YOLO(model_name)

        # Open video
        cap = cv2.VideoCapture(str(video_path))
        if not cap.isOpened():
            raise ValueError(f"Cannot open video: {video_path}")

        tracks = []
        frame_idx = 0

        # Process video
        while True:
            ret, frame = cap.read()
            if not ret:
                break

            # Run detection and tracking
            if args.tracker == "bytetrack":
                # Use YOLO's built-in tracking
                results = model.track(
                    frame,
                    persist=True,
                    conf=args.conf_threshold,
                    tracker="bytetrack.yaml",
                    verbose=False
                )
            else:
                # For other trackers, use simple detection + association
                results = model(frame, conf=args.conf_threshold, verbose=False)

            # Extract tracks
            if len(results) > 0 and results[0].boxes is not None:
                boxes = results[0].boxes

                if hasattr(boxes, 'id') and boxes.id is not None:
                    # Tracking mode - has IDs
                    box_xyxy = boxes.xyxy.cpu().numpy()
                    track_ids = boxes.id.cpu().numpy()
                    confs = boxes.conf.cpu().numpy()
                    classes = boxes.cls.cpu().numpy()

                    for box, track_id, conf, cls in zip(box_xyxy, track_ids, confs, classes):
                        tracks.append({
                            "frame_id": int(frame_idx),
                            "track_id": int(track_id),
                            "bbox": box.tolist(),
                            "score": float(conf),
                            "class_id": int(cls)
                        })
                else:
                    # Detection only mode - assign simple IDs
                    box_xyxy = boxes.xyxy.cpu().numpy()
                    confs = boxes.conf.cpu().numpy()
                    classes = boxes.cls.cpu().numpy()

                    for i, (box, conf, cls) in enumerate(zip(box_xyxy, confs, classes)):
                        tracks.append({
                            "frame_id": int(frame_idx),
                            "track_id": i,  # Simple ID assignment
                            "bbox": box.tolist(),
                            "score": float(conf),
                            "class_id": int(cls)
                        })

            frame_idx += 1

            # Progress indicator every 100 frames
            if frame_idx % 100 == 0:
                print(f"Processed {frame_idx} frames...", file=sys.stderr)

        cap.release()

        # Save results
        with open(output_path, 'w') as f:
            json.dump(tracks, f)

        print(f"Saved {len(tracks)} detections to {output_path}", file=sys.stderr)

    except Exception as e:
        print(f"Error: {e}", file=sys.stderr)
        sys.exit(1)

if __name__ == "__main__":
    main()
'''))

runner_script.chmod(0o755)
print_status("TrackLab runner script created", "SUCCESS")


[INFO] Setting up TrackLab with proper integration...
[SUCCESS] Virtual display started
[INFO] Removing existing TrackLab .venv...
[INFO] Creating TrackLab uv environment...
Using CPython 3.12.12 interpreter at: /usr/bin/python3
Creating virtual environment at: .venv
Activate with: source .venv/bin/activate
[INFO] Installing TrackLab with video extras and dependencies...
[INFO] Applying fixes to TrackLab...
[SUCCESS] TrackLab fixes applied
[INFO] Creating proper TrackLab runner script...
[SUCCESS] TrackLab runner script created


In [12]:

# Cell: Run System Evaluation (Including TrackLab)


import time
import json
import subprocess
import os

# System configurations - using the dictionary structure your run_system_on_clip expects
SYSTEM_CONFIGS = {
    "tracklab_yolov8_botsort": {
        "path": REPOS_DIR / "tracklab",
        "script": "run_tracklab.py",
        "args": ["--detector", "yolov8", "--tracker", "botsort", "--conf_threshold", "0.3"]
    },
}

position_to_number = {"start": 1, "middle": 2, "end": 3, "full": 1}

def run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path):
    """Run a tracking system on a clip"""

    output_dir = OUTPUT_DIR / video_name / "clips" / str(clip_number) / system_name
    output_dir.mkdir(parents=True, exist_ok=True)
    output_file = output_dir / "output.json"

    print_status(f"Running {system_name} on {video_name}/clip_{clip_number}...", "INFO")

    # Change to system directory
    original_dir = os.getcwd()
    os.chdir(system_config["path"])

    # Build command
    if system_name == "eagle":
        cmd = [
            "uv", "run", "--python", "python3.11", "run_eagle.py",
            "--video", str(clip_path),
            "--output", str(output_file)
        ]
    elif system_name.startswith("tracklab"):
        # Use uv run to execute our wrapper *inside* the TrackLab env
        cmd = [
            "uv", "run", "python", system_config["script"],
            "--video", str(clip_path),
            "--output", str(output_file),
        ]

        # Pass through any extra args from SYSTEM_CONFIGS (detector/tracker/conf)
        for extra in system_config.get("args", []):
            cmd.append(str(extra))
    else:
        cmd = [
            "python", system_config["script"],
            "--video", str(clip_path),
            "--output", str(output_file)
        ]

    start_time = time.time()

    try:
        result = subprocess.run(
            cmd,
            capture_output=True,
            text=True,
            timeout=600  # 10 minute timeout
        )

        elapsed = time.time() - start_time

        os.chdir(original_dir)

        # Debug output - comment out after testing
        if system_name.startswith("tracklab") and result.stderr:
            print("=== TrackLab Progress ===")
            print(result.stderr[:500])  # Show first 500 chars of stderr

        if result.returncode == 0 and output_file.exists():
            try:
                with open(output_file) as f:
                    data = json.load(f)

                # Count detections/tracks
                num_detections = len(data) if isinstance(data, list) else 0

                print_status(
                    f"{system_name} ({video_name}/clip_{clip_number}): SUCCESS - {num_detections} detections in {elapsed:.1f}s",
                    "SUCCESS"
                )
                return {"success": True, "time": elapsed, "output": str(output_file), "detections": num_detections}
            except json.JSONDecodeError:
                print_status(
                    f"{system_name} ({video_name}/clip_{clip_number}): Invalid JSON output",
                    "ERROR"
                )
                return {"success": False, "time": elapsed, "error": "Invalid JSON"}
        else:
            error_msg = result.stderr[-500:] if result.stderr else "Unknown error"  # Last 500 chars
            print_status(
                f"{system_name} ({video_name}/clip_{clip_number}): FAILED - {error_msg[:200]}",
                "ERROR"
            )
            return {"success": False, "time": elapsed, "error": error_msg}

    except subprocess.TimeoutExpired:
        os.chdir(original_dir)
        print_status(f"{system_name} ({video_name}/clip_{clip_number}): TIMEOUT", "ERROR")
        return {"success": False, "time": 600, "error": "Timeout"}

    except Exception as e:
        os.chdir(original_dir)
        print_status(f"{system_name} ({video_name}/clip_{clip_number}): EXCEPTION - {str(e)}", "ERROR")
        return {"success": False, "time": time.time() - start_time, "error": str(e)}


# Run all systems on all clips


print("\n" + "="*60)
print("STARTING EVALUATION")
print("="*60 + "\n")

all_results = {}

for video_name, clip_paths in ALL_CLIPS.items():

    print(f"\n{'='*40}")
    print(f"VIDEO: {video_name}")
    print(f"{'='*40}")

    video_results = {}

    for clip_position, clip_path in clip_paths.items():
        clip_number = position_to_number.get(clip_position, 1)

        print(f"\n{'-'*40}")
        print_status(f"Processing clip {clip_number} ({clip_position})...", "INFO")
        print(f"{'-'*40}")

        video_results[f"clip_{clip_number}"] = {}

        for system_name, system_config in SYSTEM_CONFIGS.items():
            result = run_system_on_clip(system_name, system_config, video_name, clip_number, clip_path)
            video_results[f"clip_{clip_number}"][system_name] = result

        # Print clip summary
        successful = sum(1 for r in video_results[f"clip_{clip_number}"].values() if r["success"])
        total = len(video_results[f"clip_{clip_number}"])
        print(f"\nClip {clip_number} summary: {successful}/{total} systems succeeded")

    all_results[video_name] = video_results

    # Save video summary
    summary_file = OUTPUT_DIR / video_name / "summary.json"
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)

    print_status(f"Summary saved: {summary_file}", "SUCCESS")

# Save overall summary
overall_summary = OUTPUT_DIR / "overall_summary.json"
with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)


# Print final summary


print("\n" + "="*60)
print("EVALUATION COMPLETE")
print("="*60)

# System success rates
print("\nSystem Success Rates:")
system_stats = {sys: {"success": 0, "total": 0} for sys in SYSTEM_CONFIGS.keys()}

for video_results in all_results.values():
    for clip_results in video_results.values():
        for system_name, result in clip_results.items():
            system_stats[system_name]["total"] += 1
            if result["success"]:
                system_stats[system_name]["success"] += 1

for system_name, stats in system_stats.items():
    if stats["total"] > 0:
        success_rate = (stats["success"] / stats["total"]) * 100
        print(f"  {system_name}: {stats['success']}/{stats['total']} ({success_rate:.1f}%)")

print(f"\nResults saved to: {OUTPUT_DIR}")
print(f"Overall summary: {overall_summary}")


STARTING EVALUATION


VIDEO: FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p

----------------------------------------
[INFO] Processing clip 1 (start)...
----------------------------------------
[INFO] Running tracklab_yolov8_botsort on FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p/clip_1...
=== TrackLab Progress ===
Installed 143 packages in 1.20s
Using yolov8n.pt with botsort tracker

  0%|          | 0.00/6.25M [00:00<?, ?B/s]
100%|██████████| 6.25M/6.25M [00:00<00:00, 130MB/s]
Processed 100 frames...
Processed 200 frames...
Processed 300 frames...
Processed 400 frames...
Processed 500 frames...
Processed 600 frames...
Processed 700 frames...
Processed 800 frames...
Processed 900 frames...
Processed 1000 frames...
Processed 1100 frames...
Processed 1200 frames...
Processed 1300 frames...
Saved 15310 detec
[SUCCESS] tracklab_yolov8_botsort (FULL MATCH  Croatia 1-1 Czechia  VIP Tactical Camera 720p/clip_1): SUCCESS - 15310 detections in 34.5s

Clip 1 summary: 1

In [ ]:

import cv2
import json
import numpy as np

def create_overlay_video(video_path, json_path, output_path, show_ids=True, show_confidence=False):
    """
    Create an overlayed video with tracking results

    Args:
        video_path: Path to original video
        json_path: Path to tracking JSON output
        output_path: Path for output video
        show_ids: Whether to display player IDs
        show_confidence: Whether to display confidence scores
    """
    # Load tracking data
    with open(json_path, 'r') as f:
        tracking_data = json.load(f)

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    frame_idx = 0

    # Colors for different players (cycling through a palette)
    colors = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 255, 0),  # Light Green
        (255, 128, 0),  # Light Blue
    ]

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get tracking data for this frame
        frame_key = str(frame_idx)
        if frame_key in tracking_data:
            frame_data = tracking_data[frame_key]

            # Draw each player
            if 'Coordinates' in frame_data and 'Player' in frame_data['Coordinates']:
                players = frame_data['Coordinates']['Player']

                for player_id, player_data in players.items():
                    bbox = player_data['BBox']
                    confidence = player_data.get('Confidence', 0)

                    # Get color for this player ID
                    color = colors[int(player_id) % len(colors)]

                    # Draw bounding box
                    x1, y1, x2, y2 = bbox
                    cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                    # Prepare label
                    label_parts = []
                    if show_ids:
                        label_parts.append(f"ID:{player_id}")
                    if show_confidence:
                        label_parts.append(f"{confidence:.2f}")

                    if label_parts:
                        label = " ".join(label_parts)

                        # Draw label background
                        (label_width, label_height), _ = cv2.getTextSize(
                            label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                        )
                        cv2.rectangle(
                            frame,
                            (x1, y1 - label_height - 5),
                            (x1 + label_width, y1),
                            color,
                            -1
                        )

                        # Draw label text
                        cv2.putText(
                            frame,
                            label,
                            (x1, y1 - 5),
                            cv2.FONT_HERSHEY_SIMPLEX,
                            0.5,
                            (255, 255, 255),
                            1
                        )

                    # Draw center point for transformed coordinates visualization
                    center_x = (x1 + x2) // 2
                    center_y = (y1 + y2) // 2
                    cv2.circle(frame, (center_x, center_y), 3, color, -1)

            # Add timestamp
            if 'Time' in frame_data:
                timestamp = frame_data['Time']
                cv2.putText(
                    frame,
                    f"Time: {timestamp}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    return True

print_status("Visualization function loaded", "SUCCESS")

In [ ]:
# Cell 8: Display results

import pandas as pd


print("RESULTS SUMMARY \n")


summary_data = []

for video_name, clips in all_results.items():
    for clip_key, systems in clips.items():
        for system_name, result in systems.items():
            summary_data.append({
                "Video": video_name,
                "Clip": clip_key,
                "System": system_name,
                "Status": "Valid" if result["success"] else "Invalid",
                "Time (s)": f"{result['time']:.1f}"
            })

df = pd.DataFrame(summary_data)
print(df.to_string(index=False))

total_runs = len(summary_data)
successful_runs = sum(1 for row in summary_data if row["Status"] == "Valid")


print(f"Success Rate: {successful_runs}/{total_runs} ({100*successful_runs/total_runs:.1f}%)\n")


In [ ]:

import cv2
import json
from pathlib import Path

def detect_json_format(data):
    """Detect which tracking system format the JSON is in"""
    if not data:
        return "unknown"

    first_frame_key = list(data.keys())[0]
    first_frame = data[first_frame_key]

    # Eagle format: nested dict with "Coordinates" -> "Player"
    if isinstance(first_frame, dict) and "Coordinates" in first_frame:
        if "Player" in first_frame["Coordinates"]:
            return "eagle"

    # AnshChoudhary/Darkmyter/TrackLab format: list of dicts with x, y, w, h
    if isinstance(first_frame, list) and len(first_frame) > 0:
        if "x" in first_frame[0] and "w" in first_frame[0]:
            return "xywh"

    return "unknown"

def parse_eagle_frame(frame_data):
    """Parse Eagle format frame data"""
    players = []
    if "Coordinates" in frame_data and "Player" in frame_data["Coordinates"]:
        for player_id, player_data in frame_data["Coordinates"]["Player"].items():
            bbox = player_data["BBox"]
            players.append({
                "id": int(player_id),
                "x1": bbox[0],
                "y1": bbox[1],
                "x2": bbox[2],
                "y2": bbox[3],
                "confidence": player_data.get("Confidence", 0)
            })
    return players, frame_data.get("Time", "")

def parse_xywh_frame(frame_data):
    """Parse AnshChoudhary/Darkmyter/TrackLab format frame data"""
    players = []
    if isinstance(frame_data, list):
        for player_data in frame_data:
            # Convert center x,y,w,h to corner coordinates
            center_x = player_data["x"]
            center_y = player_data["y"]
            w = player_data["w"]
            h = player_data["h"]

            x1 = int(center_x - w/2)
            y1 = int(center_y - h/2)
            x2 = int(center_x + w/2)
            y2 = int(center_y + h/2)

            players.append({
                "id": player_data["id"],
                "x1": x1,
                "y1": y1,
                "x2": x2,
                "y2": y2,
                "confidence": player_data.get("confidence", 0)
            })
    return players, ""

def create_universal_overlay_video(video_path, json_path, output_path, show_ids=True, show_confidence=False):
    """
    Create an overlayed video that works with ANY tracking system format
    """
    # Load tracking data
    with open(json_path, 'r') as f:
        tracking_data = json.load(f)

    # Detect format
    format_type = detect_json_format(tracking_data)
    print(f"    Detected format: {format_type}")

    if format_type == "unknown":
        print(f"    ✗ Unknown JSON format")
        return False

    # Open video
    cap = cv2.VideoCapture(str(video_path))
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Create video writer
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (width, height))

    # Colors for different players
    colors = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 255, 0),  # Light Green
        (255, 128, 0),  # Light Blue
        (0, 128, 255),  # Orange
        (255, 0, 128),  # Pink
    ]

    frame_idx = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        # Get tracking data for this frame
        frame_key = str(frame_idx)
        if frame_key in tracking_data:
            frame_data = tracking_data[frame_key]

            # Parse based on format
            if format_type == "eagle":
                players, timestamp = parse_eagle_frame(frame_data)
            elif format_type == "xywh":
                players, timestamp = parse_xywh_frame(frame_data)
            else:
                players, timestamp = [], ""

            # Draw each player
            for player in players:
                player_id = player["id"]
                x1, y1, x2, y2 = player["x1"], player["y1"], player["x2"], player["y2"]
                confidence = player["confidence"]

                # Get color for this player ID
                color = colors[player_id % len(colors)]

                # Draw bounding box
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)

                # Prepare label
                label_parts = []
                if show_ids:
                    label_parts.append(f"ID:{player_id}")
                if show_confidence and confidence > 0:
                    label_parts.append(f"{confidence:.2f}")

                if label_parts:
                    label = " ".join(label_parts)

                    # Draw label background
                    (label_width, label_height), _ = cv2.getTextSize(
                        label, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1
                    )
                    cv2.rectangle(
                        frame,
                        (x1, y1 - label_height - 5),
                        (x1 + label_width, y1),
                        color,
                        -1
                    )

                    # Draw label text
                    cv2.putText(
                        frame,
                        label,
                        (x1, y1 - 5),
                        cv2.FONT_HERSHEY_SIMPLEX,
                        0.5,
                        (255, 255, 255),
                        1
                    )

                # Draw center point
                center_x = (x1 + x2) // 2
                center_y = (y1 + y2) // 2
                cv2.circle(frame, (center_x, center_y), 3, color, -1)

            # Add timestamp (for Eagle format)
            if timestamp:
                cv2.putText(
                    frame,
                    f"Time: {timestamp}",
                    (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.7,
                    (255, 255, 255),
                    2
                )

            # Add frame number (for all formats)
            cv2.putText(
                frame,
                f"Frame: {frame_idx}",
                (10, height - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (255, 255, 255),
                1
            )

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()

    return True

# GENERATE OVERLAYS FOR ALL SYSTEMS

print("UNIVERSAL OVERLAY GENERATOR\n")

overlay_count = 0
failed_count = 0
system_counts = {}

for video_name, clips_data in all_results.items():
    print(f"\n📹 Video: {video_name}")

    for clip_key, systems_data in clips_data.items():
        # Reconstruct clip path
        clip_number = int(clip_key.split('_')[1])
        clip_path = None

        for pos, num in position_to_number.items():
            if num == clip_number and video_name in ALL_CLIPS and pos in ALL_CLIPS[video_name]:
                clip_path = ALL_CLIPS[video_name][pos]
                break

        if not clip_path:
            print(f" Skipping {clip_key} - clip path not found")
            continue

        print(f"\n {clip_key}:")

        for system_name, result_data in systems_data.items():
            if result_data.get('success'):
                try:
                    json_path = result_data['output']
                    output_dir = Path(json_path).parent
                    overlay_video_path = output_dir / "overlay_video.mp4"

                    print(f" {system_name}...", end=" ")

                    success = create_universal_overlay_video(
                        video_path=clip_path,
                        json_path=json_path,
                        output_path=overlay_video_path,
                        show_ids=True,
                        show_confidence=False
                    )

                    if success:
                        print(f"✓")
                        overlay_count += 1
                        system_counts[system_name] = system_counts.get(system_name, 0) + 1
                        result_data['overlay_video'] = str(overlay_video_path)
                    else:
                        print(f"✗")
                        failed_count += 1

                except Exception as e:
                    print(f"✗ Error: {e}")
                    failed_count += 1

print("SUMMARY\n")
print(f"Total overlays created: {overlay_count}")
print(f"Failed: {failed_count}")
print("\nPer System:")
for system, count in sorted(system_counts.items()):
    print(f"  • {system}: {count} video(s)")
print("="*60)

# Update summary files
for video_name, video_results in all_results.items():
    summary_file = OUTPUT_DIR / video_name / "summary.json"
    with open(summary_file, "w") as f:
        json.dump(video_results, f, indent=2)

with open(overall_summary, "w") as f:
    json.dump(all_results, f, indent=2)

print("\nSummary files updated with overlay paths!")

In [ ]:
# Cell 9: Download results

from google.colab import files
import shutil

print_status("Creating archive...", "INFO")

archive_name = "tracking_results"
archive_path = BASE_DIR / archive_name

shutil.make_archive(str(archive_path), 'zip', OUTPUT_DIR)

print_status("Downloading...", "SUCCESS")
files.download(f"{archive_path}.zip")

print_status("Complete!", "SUCCESS")